In [1]:
#load data 
import pandas as pd
import numpy as np
import requests 
import tarfile
import os 
import gensim
import gensim.downloader
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

url = "https://www.cs.jhu.edu/~mdredze/datasets/sentiment/processed_acl.tar.gz"
filename = "processed_acl"
extension = ".tar.gz"

response = requests.get(url, allow_redirects=True)

print("http response:",response.status_code,response.reason)

with open(filename+extension,"wb") as file: 
    file.write(response.content)
    
tar = tarfile.open(filename+extension, "r:gz")
tar.extractall()
tar.close()
# # merge files using shell commands 
print(os.popen("cat " + filename+"/books/all.review > mixed.txt").read())
print(os.popen("cat " + filename+"/books/positive.review >> mixed.txt").read())
# print(os.popen("cat " + filename+"/dvd/all.review >> mixed.txt").read())
# print(os.popen("cat " + filename+"/dvd/positive.review >> mixed.txt").read())



# parse data into array for word 2 vect 
dataArray = []
labelArray = []

with open("mixed.txt", "r") as datafile: 
    lines = datafile.readlines()
    for e in lines:
        lineList= []
        splitLine = e.split()
        label = splitLine.pop(-1)
        for i in splitLine:
            dictSplit = i.split(':')
            lineList.append(dictSplit[0])
        dataArray.insert(-1, lineList)
        labelArray.insert(-1, (label.split(':')[1]))

#download word2vec model          
word2vec = gensim.downloader.load('fasttext-wiki-news-subwords-300')


# create mean embedding 
embed = np.array([ np.mean([word2vec[w] for w in words if w in word2vec], axis=0) for words in dataArray])

X_train, X_test, y_train, y_test = train_test_split(embed, labelArray, test_size=0.2, random_state=42)

text_classifier = RandomForestClassifier(n_estimators=700, random_state=42)
text_classifier.fit(X_train, y_train)

predictions = text_classifier.predict(X_test)

from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

print(confusion_matrix(y_test,predictions))
print(classification_report(y_test,predictions))
print(accuracy_score(y_test, predictions))



/home/tom/.local/lib/python3.8/site-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


http response: 200 OK


[[200]]
              precision    recall  f1-score   support

    positive       1.00      1.00      1.00       200

    accuracy                           1.00       200
   macro avg       1.00      1.00      1.00       200
weighted avg       1.00      1.00      1.00       200

1.0


In [19]:
# random forest 
from math import log 
from collections import defaultdict, Counter
from functools import partial

# calculate entropy 
def entropy(probabilities):
    return sum(-p * log(p,2) for p in probabilities if p )

def classProbabilities(classes):
    return [count / len(classes) for count in Counter(classes).values()]

def get_entropy(data):
    return entropy(classProbabilities([label for _, label in data]))

def calcPartEntropy(set):
    total = sum(len(subset) for subset in set)
    
    return sum( get_entropy(subset) * len(subset) / total for subset in set)

def partitionSet(set, feature):
    groups = defaultdict(list)
    for subset in set:
        if feature in subset[0]:
            key = subset[0][feature]
            groups[key].append(subset)
    return groups 

def partittion_entropy(set,feature):
    partitions = partitionSet(set,feature)
    return calcPartEntropy(partitions.values())
    
    
print(partittion_entropy(dataArray,"shallow"))
    

# build tree 

def build_tree(input_set, feature_split = None):
    #print(len(input_set))
    # if no split feature selected the first 
    #print(feature_split)
    if not feature_split:
        feature_split = input_set[0][0].keys()
    #print(len(feature_split))
    # check for leaves 
    setSize = len(input_set)
    positive_count = 0 
    negative_count = 0 
    for e in input_set:
        if e[1] == "positive":
            positive_count+=1 
        else:
            negative_count+=1
    print("positive count:",positive_count,"negative count:",negative_count)
#     if positive_count == 0:
#         return "positive"
#     if negative_count == 0: 
#         return "negative"
    
    if not feature_split:
        if positive_count > negative_count:
            return "positive"
        else:
            return "negative"
        
    best_feature = min(feature_split, key = partial(partittion_entropy,input_set))
    #print (feature_split)
    
    partitions = partitionSet(input_set, best_feature)
    
    #print(partitions)
    new_split_features = [e for e in feature_split if e != best_feature]
    
    #print(new_split_features)
    
    subTrees = {attribute_value : build_tree(subset,new_split_features)
               for attribute_value, subset in partitions.items()}
    print(subTrees)
    if not subTrees:
         if positive_count > negative_count:
            return "positive"
         else:
            return "negative"
    
    return (best_feature, subTrees)

tree = build_tree(dataArray,list(keySet))
print(tree)
        

0


NameError: name 'keySet' is not defined